<a href="https://colab.research.google.com/github/larsmoan/PatternAnalysis-2023/blob/main/lesion_detection_larsmoan/isic_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone "https://github.com/larsmoan/PatternAnalysis-2023.git"
!cd PatternAnalysis-2023

Cloning into 'PatternAnalysis-2023'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 65 (delta 24), reused 63 (delta 24), pack-reused 0
Receiving objects: 100% (65/65), 83.76 KiB | 3.81 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [2]:
!cd PatternAnalysis-2023/ && git submodule update --init

Submodule 'lesion_detection_larsmoan/yolov7' (https://github.com/WongKinYiu/yolov7) registered for path 'lesion_detection_larsmoan/yolov7'
Cloning into '/content/PatternAnalysis-2023/lesion_detection_larsmoan/yolov7'...
Submodule path 'lesion_detection_larsmoan/yolov7': checked out '84932d70fb9e2932d0a70e4a1f02a1d6dd1dd6ca'


In [4]:
import os
os.chdir('/content/PatternAnalysis-2023/lesion_detection_larsmoan/')
!ls

dataset.py	     hyp.scratch.p5.yaml  predict.py	       slurm.sh  yolov7_isic_cfg.yaml
dataset_utils.ipynb  hyp.scratch.p6.yaml  README.md	       utils.py
downsample.py	     modules.py		  run_custom_train.sh  yolov7


In [5]:
import gdown
import os
import zipfile
def download_and_unzip(folder_path, download_url):
    """
    Checks if a folder exists at the specified path.
    If not, downloads a zip file from the provided URL using gdown and unzips it.

    Parameters:
    - folder_path: Path to the folder you want to check.
    - download_url: The gdown URL of the zip file to download if the folder doesn't exist.
    """

    # Check if folder exists
    if not os.path.exists(folder_path):
        print(f"Folder {folder_path} does not exist. Downloading the zip file...")

        # Download zip file using gdown
        output = "temp_download.zip"
        gdown.download(download_url, output, quiet=False)


        # Unzip the downloaded file
        print("Unzipping the downloaded file...")
        with zipfile.ZipFile(output, 'r') as zip_ref:
            zip_ref.extractall(folder_path)

        # Delete the downloaded zip file
        os.remove(output)
        print(f"Files have been extracted to {folder_path}")
    else:
        print(f"Folder {folder_path} already exists.")

In [6]:
dataset_folder = "data"
url = "https://drive.google.com/uc?id=19HcgRBuXyhzxsukE2jHPnBkC0hEk_aMJ"
download_and_unzip(dataset_folder, url)

Folder data does not exist. Downloading the zip file...


Downloading...
From: https://drive.google.com/uc?id=19HcgRBuXyhzxsukE2jHPnBkC0hEk_aMJ
To: /content/PatternAnalysis-2023/lesion_detection_larsmoan/temp_download.zip
100%|██████████| 71.8M/71.8M [00:00<00:00, 76.7MB/s]


Unzipping the downloaded file...
Files have been extracted to data


In [17]:
!python3 yolov7/train.py --epochs 100 --workers 32 --device 0 --batch-size 16 --data data/ISIC_2017_downsampled/isic.yaml --img 256 256 --cfg yolov7_isic_cfg.yaml --weights '' --name yolov7_isic --hyp hyp.scratch.p5.yaml

2023-10-16 04:52:10.996744: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-16 04:52:11.936076: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
YOLOR 🚀 v0.1-126-g84932d7 torch 2.0.1+cu118 CUDA:0 (Tesla T4, 15101.8125MB)

Namespace(weights='', cfg='yolov7_isic_cfg.yaml', data='data/ISIC_2017_downsampled/isic.yaml', hyp='hyp.scratch.p5.yaml', epochs=100, batch_size=16, img_size=[256, 256], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='0', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=32, project='runs/train', entity=None, name='yolov7_isic', exist_ok=Fa

In [20]:
!python yolov7/detect.py --weights runs/train/yolov7_isic9/weights/best.pt --conf 0.2 --img-size 640 --source data/ISIC_2017_downsampled/test_downsampled

Namespace(weights=['runs/train/yolov7_isic9/weights/best.pt'], source='data/ISIC_2017_downsampled/test_downsampled', img_size=640, conf_thres=0.2, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-126-g84932d7 torch 2.0.1+cu118 CUDA:0 (Tesla T4, 15101.8125MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
Model Summary: 314 layers, 36492560 parameters, 6194944 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignor